In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [32]:
train_dir=Path("etsydata/train/")




In [33]:
train_dir

WindowsPath('etsydata/train')

In [79]:
test_dir=Path("etsydata/test/")




In [11]:
def read_data(filenames,columns=None):
    """ Read parquet data from mutiple files """

    dataframes = [

        pd.read_parquet(filenames,columns=columns)
    ]

    return pd.concat(dataframes)
columns=["primary_color_id","secondary_color_id","title","description","tags"]



In [34]:
df_train = read_data(train_dir,columns=columns)

In [13]:
from sklearn.model_selection import train_test_split

df_training, df_validation = train_test_split(
    df_train,
    test_size=0.3,
    stratify=df_train['secondary_color_id'],
    random_state=42)
df_training['combined_text'] = df_training['title'].fillna('') + ' ' + df_training['description'].fillna('') + ' ' + df_training['tags'].fillna('')
df_validation['combined_text'] = df_validation['title'].fillna('') + ' ' + df_validation['description'].fillna('') + ' ' + df_validation['tags'].fillna('')

In [14]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lower case
    text = text.lower().strip()
    return text


df_training['combined_text'] = df_training['combined_text'].apply(clean_text)
df_validation['combined_text'] = df_validation['combined_text'].apply(clean_text)


tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)


X_train_tfidf = tfidf_vectorizer.fit_transform(df_training['combined_text'])
X_val_tfidf = tfidf_vectorizer.transform(df_validation['combined_text'])


In [15]:
from sklearn.ensemble import RandomForestClassifier

rf_clf_optimized = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,  # 'None' means unlimited depth
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight='balanced',  # Adjust weights inversely proportional to class frequencies
    random_state=42
)

In [16]:
rf_clf_optimized.fit(X_train_tfidf, df_training['secondary_color_id'])

RandomForestClassifier(class_weight='balanced', n_estimators=300,
                       random_state=42)

In [36]:
pred_secondary=rf_clf_optimized.predict(X_val_tfidf)

In [37]:
from sklearn.metrics import f1_score
print("F1 Score Random Forest:", f1_score(df_validation['secondary_color_id'], pred_secondary, average='weighted'))

F1 Score Random Forest: 0.35341145500987053


In [49]:
from sklearn import metrics

In [50]:
print(metrics.classification_report(df_validation['secondary_color_id'], pred_secondary))

              precision    recall  f1-score   support

           0       0.47      0.20      0.28      3096
           1       0.36      0.41      0.38      8931
           2       0.34      0.21      0.26      5264
           3       0.46      0.26      0.33       839
           4       0.37      0.37      0.37      4843
           5       0.54      0.33      0.41      1345
           6       0.51      0.31      0.38       546
           7       0.45      0.57      0.50      4518
           9       0.35      0.25      0.30      4680
          10       0.32      0.24      0.28      1841
          11       0.36      0.30      0.33      3853
          12       0.30      0.23      0.26      1712
          13       0.39      0.27      0.32      1826
          14       0.35      0.24      0.29      4575
          15       0.75      0.19      0.31       352
          16       0.48      0.53      0.50      4054
          17       0.30      0.55      0.39     11200
          18       0.36    

In [42]:
dfpredctionsec=pd.DataFrame()

In [44]:
dfpredctionsec["secondary_color_id"]=pred_secondary

In [45]:
import pyarrow as pa
import pyarrow.parquet as pq

In [61]:
dfpredctionsec.to_csv('AArekh_sec.csv', index=False)

In [58]:
dfpredctionsec

,secondary_color_id
0,7
1,2
2,4
3,17
4,17
...,...
68883,17
68884,0
68885,17
68886,4


In [22]:
from sklearn.ensemble import RandomForestClassifier

rf_clf_optimized_ = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,  # 'None' means unlimited depth
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight='balanced',  # Adjust weights inversely proportional to class frequencies
    random_state=42
)
rf_clf_optimized_.fit(X_train_tfidf, df_training['primary_color_id'])

RandomForestClassifier(class_weight='balanced', n_estimators=300,
                       random_state=42)

In [47]:
predtop=rf_clf_optimized_.predict(X_val_tfidf)

In [51]:
from sklearn.metrics import f1_score
print("F1 Score Random Forest:", f1_score(df_validation['primary_color_id'], predtop, average='weighted'))

F1 Score Random Forest: 0.4802996906340763


In [52]:
print(metrics.classification_report(df_validation['secondary_color_id'], predtop))

              precision    recall  f1-score   support

           0       0.22      0.13      0.16      3096
           1       0.24      0.33      0.27      8931
           2       0.20      0.39      0.26      5264
           3       0.26      0.13      0.17       839
           4       0.25      0.34      0.29      4843
           5       0.28      0.23      0.25      1345
           6       0.28      0.25      0.27       546
           7       0.34      0.21      0.26      4518
           9       0.26      0.19      0.22      4680
          10       0.25      0.18      0.21      1841
          11       0.27      0.34      0.30      3853
          12       0.25      0.24      0.24      1712
          13       0.31      0.14      0.20      1826
          14       0.25      0.23      0.24      4575
          15       0.11      0.02      0.03       352
          16       0.36      0.30      0.33      4054
          17       0.23      0.20      0.22     11200
          18       0.23    

In [56]:
dfpredctionprim.to_csv('AArekh_prim.csv', index=False)

In [64]:
df_top=pd.read_csv('aarekh_etsy.csv')

In [70]:
bottom_values=df_bot.values

In [71]:
bottom_values

array([[1279],
       [6407],
       [6404],
       ...,
       [9183],
       [9183],
       [9183]], dtype=int64)

In [72]:
df_top['bottom_category_id']=bottom_values

In [73]:
df_top

,top_category_id,bottom_category_id
0,8,1279
1,8,6407
2,6,6404
3,6,6404
4,6,6404
...,...,...
25509,10,9183
25510,10,9183
25511,10,9183
25512,10,9183


In [67]:
df_bot=pd.read_csv('aarekh_etsy_bottom.csv')

In [29]:
 from sklearn.ensemble import RandomForestClassifier

rf_clf_optimized_2 = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,  # 'None' means unlimited depth
    min_samples_split=4,
    max_features='log2',
    min_samples_leaf=2,
    class_weight='balanced',  # Adjust weights inversely proportional to class frequencies
    random_state=42
)
rf_clf_optimized_2.fit(X_train_tfidf, df_training['primary_color_id'])

RandomForestClassifier(class_weight='balanced', max_features='log2',
                       min_samples_leaf=2, min_samples_split=4,
                       n_estimators=300, random_state=42)

In [30]:
preds=rf_clf_optimized_2.predict(X_val_tfidf)

In [31]:
print("F1 Score Random Forest:", f1_score(df_validation['primary_color_id'], preds, average='weighted'))

F1 Score Random Forest: 0.44096207072134025


In [77]:
columns=["title","description","tags","type","product_id"]

In [80]:
df_test = read_data(test_dir,columns=columns)

In [81]:
df_test['combined_text'] = df_test['title'].fillna('') + ' ' + df_test['description'].fillna('') + ' ' + df_test['tags'].fillna('')


In [82]:
df_test['combined_text'] = df_test['combined_text'].apply(clean_text)

In [83]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

# Fit and transform the combined_text
X_test_tfidf = tfidf_vectorizer.fit_transform(df_test['combined_text'])

In [86]:
predtest_prim=rf_clf_optimized_.predict(X_test_tfidf)

In [87]:
predtest_sec=rf_clf_optimized.predict(X_test_tfidf)

In [92]:
df_top["primary_color_id"]=predtest_prim

In [93]:
df_top["secondary_color_id"]=predtest_sec

In [95]:
df_top["product_id"]=df_test['product_id']

In [97]:
df=df_top

In [98]:
df

,top_category_id,bottom_category_id,primary_color_id,secondary_color_id,product_id
0,8,1279,12,12,661373440
1,8,6407,2,17,1501009290
2,6,6404,9,9,1105447030
3,6,6404,12,12,1140081090
4,6,6404,1,17,793448890
...,...,...,...,...,...
25509,10,9183,12,12,869022010
25510,10,9183,12,12,954065860
25511,10,9183,12,12,1417634130
25512,10,9183,12,12,942614980


In [99]:
df.to_csv('Aarekh_etsy.csv', index=False)

In [100]:
table = pa.Table.from_pandas(df)

In [101]:
table

pyarrow.Table
top_category_id: int64
bottom_category_id: int64
primary_color_id: int64
secondary_color_id: int64
product_id: int64
----
top_category_id: [[8,8,6,6,6,...,10,10,10,10,10]]
bottom_category_id: [[1279,6407,6404,6404,6404,...,9183,9183,9183,9183,9183]]
primary_color_id: [[12,2,9,12,1,...,12,12,12,12,12]]
secondary_color_id: [[12,17,9,12,17,...,12,12,12,12,12]]
product_id: [[661373440,1501009290,1105447030,1140081090,793448890,...,869022010,954065860,1417634130,942614980,1200453060]]

In [102]:
parquet_file_path = "Predictions_23262763.parquet"

In [104]:
output=pq.write_table(table, parquet_file_path)

In [105]:
output